In [ ]:
!pip install autogluon
!pip install mxnet~=1.9

In [ ]:
import random
import os
import pandas as pd
import numpy as np
from tqdm.auto import tqdm
from sklearn.preprocessing import LabelEncoder
from autogluon.common import space
from tqdm import tqdm

import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
from autogluon.timeseries import TimeSeriesDataFrame, TimeSeriesPredictor
from sklearn.preprocessing import LabelEncoder

In [ ]:
path = './data/'

In [ ]:
train = pd.read_csv(path + 'train.csv')
test = pd.read_csv(path + 'test.csv')
submission = pd.read_csv(path + 'sample_submission.csv')

train['timestamp'] = pd.to_datetime(train['timestamp'])
test['timestamp'] = pd.to_datetime(test['timestamp'])

train['item_id'] = train['item'] + train['corporation'] + train['location']
test['item_id'] = test['item'] + test['corporation'] + test['location']

train.rename(columns={'price(원/kg)': 'target'}, inplace=True)
train['log_target'] = np.where(train['target'] == 0, 0, np.log(train['target']))

le = LabelEncoder()
train['item_id'] = le.fit_transform(train['item_id'])
test['item_id'] = le.transform(test['item_id'])

for df in [train, test]:
    df['year'] = df['timestamp'].dt.year
    df['month'] = df['timestamp'].dt.month
    df['day'] = df['timestamp'].dt.day
    df['weekday'] = df['timestamp'].dt.weekday
    df['week'] = df['timestamp'].dt.isocalendar().week
    df['quarter'] = df['timestamp'].dt.quarter
    df['days_in_month'] = df['timestamp'].dt.days_in_month
    df['is_month_start'] = df['timestamp'].dt.is_month_start
    df['is_month_end'] = df['timestamp'].dt.is_month_end
    df['is_quarter_start'] = df['timestamp'].dt.is_quarter_start
    df['is_quarter_end'] = df['timestamp'].dt.is_quarter_end
    df['is_year_start'] = df['timestamp'].dt.is_year_start
    df['is_year_end'] = df['timestamp'].dt.is_year_end
    df['is_leap_year'] = df['timestamp'].dt.is_leap_year
    df['dayofyear'] = df['timestamp'].dt.dayofyear

/usr/local/lib/python3.9/dist-packages/pandas/core/arraylike.py:405: RuntimeWarning: divide by zero encountered in log
  result = getattr(ufunc, method)(*inputs, **kwargs)


In [ ]:
train = train[['item_id', 'timestamp', 'target', 'log_target']]
test = test[['item_id', 'timestamp']]
train = train[['item_id', 'timestamp', 'target']]
test = test[['item_id', 'timestamp']]

In [ ]:
# train = train[['item_id', 'timestamp', 'target', 'log_target', 'year', 'month', 'day',
#                'weekday', 'week', 'quarter', 'days_in_month',
#        'is_month_start', 'is_month_end', 'is_quarter_start', 'is_quarter_end',
#        'is_year_start', 'is_year_end', 'is_leap_year', 'dayofyear']]
# test = test[['item_id', 'timestamp', 'year', 'month', 'day',
#              'weekday', 'week', 'quarter', 'days_in_month',
#        'is_month_start', 'is_month_end', 'is_quarter_start', 'is_quarter_end',
#        'is_year_start', 'is_year_end', 'is_leap_year', 'dayofyear']]

In [ ]:
train = train.set_index(['item_id', 'timestamp'])
test = test.set_index(['item_id', 'timestamp'])

In [ ]:
# predictor = TimeSeriesPredictor(prediction_length=28, target='target', eval_metric="RMSE", num_val_windows=7, known_covariates_names=['year', 'month', 'day',
#         'weekday', 'week', 'quarter', 'days_in_month',
#        'is_month_start', 'is_month_end', 'is_quarter_start', 'is_quarter_end',
#        'is_year_start', 'is_year_end', 'is_leap_year', 'dayofyear']).fit(train, presets='best_quality')

# predictor = TimeSeriesPredictor(prediction_length=28, target='target', eval_metric="RMSE").fit(train, presets='best_quality')
predictor = TimeSeriesPredictor(prediction_length=28, target='target').fit(
                                                                            train,
                                                                            num_val_windows=6,
                                                                            # presets='best_quality',
                                                                            hyperparameter_tune_kwargs={
                                                                                    "scheduler": "local",
                                                                                    "searcher": "random",
                                                                                    "num_trials": 30,
                                                                                },
                                                                            hyperparameters={
                                                                                "SeasonalNaive": {
                                                                                    "seasonal_period": space.Int(91,183,365)
                                                                                },
                                                                                "AutoETS": {
                                                                                    "seasonal_period": space.Int(91,183,365)
                                                                                },
                                                                                "Theta": {
                                                                                    "seasonal_period": space.Int(91,183,365)
                                                                                },
                                                                            #     "RecursiveTabular": {
                                                                            #         "lags": space.Int(91,183,365)
                                                                            #     },
                                                                                "DeepAR": {
                                                                                    "context_length": space.Int(28, 365),                                                                                    ""
                                                                                    "num_layers": space.Int(2, 8),
                                                                                    "hidden_size": space.Int(20, 100),
                                                                                    "dropout_rate": space.Categorical(0.1, 0.3),
                                                                                },
                                                                            #     "AutoARIMA": {
                                                                            #         "seasonal_period": space.Int(91,183,365)
                                                                            #     },
                                                                            #     "DirectTabular": {
                                                                            #     },
                                                                            #     "PatchTST": {
                                                                            #         "context_length": space.Int(28, 365),
                                                                            #         "patch_len": space.Int(28,56),
                                                                            #         "stride": space.Int(14,28),
                                                                            #         "d_model": space.Int(56,112),
                                                                            #         "n_head": space.Int(4,8),
                                                                            #     },
                                                                            #     "TemporalFusionTransformer": {
                                                                            #         "context_length": space.Int(28, 365),
                                                                            #         "num_layers": space.Int(2, 8),
                                                                            #         "hidden_size": space.Int(20, 100),
                                                                            #         "dropout_rate": space.Categorical(0.1, 0.3),
                                                                            #     },
                                                                            },
                                                                           )

In [ ]:
y_pred = predictor.predict(train)
y_pred.reset_index(drop=True, inplace=True)
submission['answer'] = y_pred['mean']
submission['answer'] = np.where(submission['answer'] < 0, 0, submission['answer'])
sunday_ids_march_2023 = [f"_202303{day:02}" for day in [5, 12, 19, 26]]
for sunday_id in sunday_ids_march_2023:
    submission.loc[submission['ID'].str.contains(sunday_id), 'answer'] = 0

submission['answer'] = np.floor(submission['answer'])
submission.to_csv(path + 'autogloun_hype_final.csv', index=False)